<a href="https://colab.research.google.com/github/rizkydianputra/-MACHINE-LEARNING/blob/main/Distributed_XGBoost_with_Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ray


     |████████████████████████████████| 54.7 MB 34 kB/s 
     |████████████████████████████████| 72 kB 486 kB/s 


In [2]:
pip install xgboost_ray


     |████████████████████████████████| 136 kB 5.1 MB/s 


In [3]:
pip install sklearn


In [4]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(num_actors=2, cpus_per_actor=1))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-13 04:30:32,603	INFO main.py:971 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
2021-11-13 04:30:34,847	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(pid=291) [04:30:35] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pid=291) [04:30:35] Tree method is automatically selected to be 'approx' for distributed training.
(pid=315) [04:30:35] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pi

Final training error: 0.0053


In [5]:
from xgboost_ray import RayDMatrix, RayParams, predict
from sklearn.datasets import load_breast_cancer
import xgboost as xgb

data, labels = load_breast_cancer(return_X_y=True)

dpred = RayDMatrix(data, labels)

bst = xgb.Booster(model_file="model.xgb")
pred_ray = predict(bst, dpred, ray_params=RayParams(num_actors=2))

print(pred_ray)

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-13 04:30:49,524	INFO main.py:1535 -- [RayXGBoost] Created 2 remote actors.
2021-11-13 04:30:51,730	INFO main.py:1552 -- [RayXGBoost] Starting XGBoost prediction.


[0.07511145 0.04715708 0.02642454 0.11464816 0.08779665 0.19609766
 0.02642454 0.03200296 0.03968306 0.06930447 0.09290799 0.02642454
 0.03476055 0.05490047 0.05012454 0.02642454 0.03337562 0.02642454
 0.02642454 0.9528658  0.9751126  0.9751126  0.07511145 0.02642454
 0.02642454 0.04102892 0.03118352 0.02642454 0.02642454 0.08779665
 0.02642454 0.03476055 0.02642454 0.02642454 0.02642454 0.02642454
 0.0665335  0.9690314  0.33619487 0.0696831  0.5729184  0.15662025
 0.03451282 0.03200296 0.12301622 0.04102892 0.9751126  0.04096693
 0.97315884 0.9234187  0.9751126  0.9751126  0.9751126  0.03059936
 0.09855346 0.9751126  0.02642454 0.03755146 0.9751126  0.97315884
 0.9751126  0.97315884 0.02642454 0.9751126  0.03120249 0.03120249
 0.97315884 0.9751126  0.8630876  0.9751126  0.02642454 0.9751126
 0.02642454 0.18486539 0.9751126  0.02642454 0.9684864  0.07511145
 0.04528174 0.9751126  0.97315884 0.8423582  0.02642454 0.02642454
 0.9751126  0.02642454 0.18525673 0.02642454 0.9610139  0.91992

In [6]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

num_actors = 1
num_cpus_per_actor = 1

ray_params = RayParams(
    num_actors=num_actors, cpus_per_actor=num_cpus_per_actor)

def train_model(config):
    train_x, train_y = load_breast_cancer(return_X_y=True)
    train_set = RayDMatrix(train_x, train_y)

    evals_result = {}
    bst = train(
        params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)
    bst.save_model("model.xgb")

from ray import tune

# Specify the hyperparameter search space.
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

# Make sure to use the `get_tune_resources` method to set the `resources_per_trial`
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())
print("Best hyperparameters", analysis.best_config)

2021-11-13 04:31:16,477	WARNING function_runner.py:564 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-11-13 04:31:16,500	INFO logger.py:606 -- pip install 'ray[tune]' to see TensorBoard files.
2021-11-13 04:31:16,502	WARNING callback.py:115 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,eta,max_depth,subsample
train_model_8a8e0_00000,PENDING,,0.000131344,3,0.659906
train_model_8a8e0_00001,PENDING,,0.000142563,5,0.799691
train_model_8a8e0_00002,PENDING,,0.00729575,4,0.582304
train_model_8a8e0_00003,PENDING,,0.000235258,7,0.606718


(pid=400) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
(pid=400)   warnings.warn(LEGACY_WARNING)
(pid=400) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:422: UserWarning: `num_actors` in `ray_params` is smaller than 2 (1). XGBoost will NOT be distributed!
(pid=400)   f"`num_actors` in `ray_params` is smaller than 2 "
(pid=400) 2021-11-13 04:31:18,905	INFO main.py:971 -- [RayXGBoost] Created 1 new actors (1 total actors). Waiting until actors are ready for training.
(pid=400) 2021-11-13 04:31:21,621	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.


Trial name,status,loc,eta,max_depth,subsample
train_model_8a8e0_00000,RUNNING,172.28.0.2:400,0.000131344,3,0.659906
train_model_8a8e0_00001,PENDING,,0.000142563,5,0.799691
train_model_8a8e0_00002,PENDING,,0.00729575,4,0.582304
train_model_8a8e0_00003,PENDING,,0.000235258,7,0.606718


Result for train_model_8a8e0_00000:
  date: 2021-11-13_04-31-22
  done: false
  experiment_id: dd9bdfca1420455fa56e68df09185bac
  hostname: 1443233f5174
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 400
  time_since_restore: 3.8423569202423096
  time_this_iter_s: 3.8423569202423096
  time_total_s: 3.8423569202423096
  timestamp: 1636777882
  timesteps_since_restore: 0
  train-error: 0.047452
  train-logloss: 0.693039
  training_iteration: 1
  trial_id: 8a8e0_00000
  
Result for train_model_8a8e0_00000:
  date: 2021-11-13_04-31-22
  done: true
  experiment_id: dd9bdfca1420455fa56e68df09185bac
  experiment_tag: 0_eta=0.00013134,max_depth=3,subsample=0.65991
  hostname: 1443233f5174
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 400
  time_since_restore: 3.9377543926239014
  time_this_iter_s: 0.006201267242431641
  time_total_s: 3.9377543926239014
  timestamp: 1636777882
  timesteps_since_restore: 0
  train-error: 0.02109
  train-logloss: 0.692037
  training_iter

...

(pid=712) 2021-11-13 04:31:39,554	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.


Result for train_model_8a8e0_00003:
  date: 2021-11-13_04-31-40
  done: false
  experiment_id: c2973e6c3f7b4df19e29f0ed846c0888
  hostname: 1443233f5174
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 712
  time_since_restore: 3.8155412673950195
  time_this_iter_s: 3.8155412673950195
  time_total_s: 3.8155412673950195
  timestamp: 1636777900
  timesteps_since_restore: 0
  train-error: 0.050967
  train-logloss: 0.692951
  training_iteration: 1
  trial_id: 8a8e0_00003
  
Result for train_model_8a8e0_00003:
  date: 2021-11-13_04-31-40
  done: true
  experiment_id: c2973e6c3f7b4df19e29f0ed846c0888
  experiment_tag: 3_eta=0.00023526,max_depth=7,subsample=0.60672
  hostname: 1443233f5174
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 712
  time_since_restore: 3.8856687545776367
  time_this_iter_s: 0.005532264709472656
  time_total_s: 3.8856687545776367
  timestamp: 1636777900
  timesteps_since_restore: 0
  train-error: 0.024605
  train-logloss: 0.691156
  training_ite

Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_8a8e0_00000,TERMINATED,172.28.0.2:400,0.000131344,3,0.659906,10,3.93775,0.692037,0.02109
train_model_8a8e0_00001,TERMINATED,172.28.0.2:504,0.000142563,5,0.799691,10,3.92951,0.691906,0.010545
train_model_8a8e0_00002,TERMINATED,172.28.0.2:608,0.00729575,4,0.582304,10,3.90039,0.635375,0.026362
train_model_8a8e0_00003,TERMINATED,172.28.0.2:712,0.000235258,7,0.606718,10,3.88567,0.691156,0.024605


(pid=712) 2021-11-13 04:31:40,715	INFO main.py:1498 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.94 seconds (1.16 pure XGBoost training time).
2021-11-13 04:31:40,874	INFO tune.py:630 -- Total run time: 24.40 seconds (24.22 seconds for the tuning loop).


Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.00014256318047081568, 'subsample': 0.7996911868587437, 'max_depth': 5}
